# Giới thiệu
### Bài toán: Dùng CNN để phân loại ảnh chó mèo
### Yêu cầu:
#### 1. Về cấu trúc model: phát triển 1 mô hình CNN phù hợp, sử dụng MaxPooling, Convolution layers, Dense layers, ReLU 
#### 2. chia train/val/test theo tỉ lệ 56/14/30
#### 3. Áp dụng các kỹ thuật Augmentation, Adam Optimizer và LR Scheduler
#### 4. Kiểm tra kết quả trên tập train/val/test bằng accuracy và F1



In [3]:
!pip install clearml


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
from clearml import Task


ModuleNotFoundError: No module named 'clearml'

In [ ]:
from tensorboardX import SummaryWriter

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
# kaggle
%env CLEARML_API_ACCESS_KEY=0YZWWGF2QG4JX723MRO5
%env CLEARML_API_SECRET_KEY=jLuecPIHXr2wVtvGHExy9eOnF5RnHXj4osTd6v3hHY5BR7o2Hb

In [ ]:
# init logger:
task = Task.init(project_name='CatvsDog-CNN', task_name='PyTorch CatvsDog Simple CNN')
writer = SummaryWriter('runs')

## Flower Recognition Using Transfer Learning

### 1. Let's first view the distribution of the data

In [ ]:
import os
paths = []
labels = []
for animal in os.listdir("/kaggle/input/cat-and-dog/training_set/training_set"):
    path_data = os.listdir(os.path.join("/kaggle/input/cat-and-dog/training_set/training_set",animal))
    cur_path = os.path.join("/kaggle/input/cat-and-dog/training_set/training_set",animal)
    paths.extend([os.path.join(cur_path,img) for img in path_data])
    labels.extend([animal]*len(path_data))

In [ ]:
import pandas as pd
data = pd.DataFrame({"path":paths,"label":labels})

In [ ]:
data.label.hist()

### Is the data imbalance?

The data is not imbalanced!

### 2. Data Loader and Custom Data Class

## Trong Pytorch, có nhiều cách để tạo 1 dataset và dataloader. Tuy nhiên, cách linh hoạt nhất vẫn là tạo 1 custom dataset class
## Một custom dataset class có cấu trúc như sau:
### __init__(params) : Method dùng để khởi tạo dataset class với các cấu trúc cần thiết
### __getitem__(idx): Method dùng để trả về 1 mẫu data (hình) + label có index là idx
### __len__() : trả về số mẫu trong dataset

In [ ]:
from PIL import Image
import os
import torch
from sklearn.model_selection import train_test_split

# define a data class
class ClassificationDataset:
    def __init__(self, data, data_path, transform, training=True):
        """Define the dataset for classification problems

        Args:
            data ([dataframe]): [a dataframe that contain 2 columns: image name and label]
            data_path ([str]): [path/to/folder that contains image file]
            transform : [augmentation methods and transformation of images]
            training (bool, optional): []. Defaults to True.
        """
        self.data = data
        self.imgs = data["path"].unique().tolist()
        self.data_path = data_path
        self.training = training
        self.transform = transform

    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.data_path, self.data.iloc[idx, 0]))
        label = self.data.iloc[idx, 1]
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.imgs)


def make_loader(dataset, train_batch_size, validation_split=0.2):
    """make dataloader for pytorch training

    Args:
        dataset ([object]): [the dataset object]
        train_batch_size ([int]): [training batch size]
        validation_split (float, optional): [validation ratio]. Defaults to 0.2.

    Returns:
        [type]: [description]
    """
    # number of samples in train and test set
    train_len = int(len(dataset) * (1 - validation_split))
    test_len = len(dataset) - train_len
    train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
    # create train_loader
    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=train_batch_size, shuffle=True,
    )
    # create test_loader
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False,)
    return train_loader, test_loader


def data_split(data, test_size):
    x_train, x_test, y_train, y_test = train_test_split(
        data, data["label"], test_size=test_size, stratify = data.iloc[:,1]
    )
    return x_train, x_test, y_train, y_test

### Define Data Augmentation Strategy

In [ ]:
import torchvision.transforms as transform
import torchvision
transformer = transform.Compose([
                           transform.Resize((224, 224)),
                           transform.RandomRotation(5),
                        transform.RandomHorizontalFlip(),
                           transform.RandomAffine(degrees=11, translate=(0.1,0.1), scale=(0.8,0.8)),
                           transform.ToTensor(),
                            transform.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

### Train test split

In [ ]:
x_train = data.copy()

In [ ]:
paths = []
labels = []
for animal in os.listdir("/kaggle/input/cat-and-dog/test_set/test_set"):
    path_data = os.listdir(os.path.join("/kaggle/input/cat-and-dog/test_set/test_set",animal))
    cur_path = os.path.join("/kaggle/input/cat-and-dog/test_set/test_set",animal)
    paths.extend([os.path.join(cur_path,img) for img in path_data])
    labels.extend([animal]*len(path_data))
x_test = pd.DataFrame({"path":paths,"label":labels})

In [ ]:
x_test

In [ ]:
x_test.label = x_test.label.map({'cats':0,'dogs':1})
x_train.label = x_train.label.map({'cats':0,'dogs':1})

In [ ]:
x_test = x_test[x_test["path"].str.contains("_DS_Store")==False]
x_train = x_train[x_train["path"].str.contains("_DS_Store")==False]

## Implement data loaders for train/val/test

## Data Loader: Tạo 1 loader với các đặc điểm:
#### Cho phép iterate qua từng mẫu trong dataset
#### tùy chỉnh sampler
#### tùy chỉnh batch size
#### tùy chỉnh collate_fn (sẽ học trong buổi 10)

In [ ]:
dataset = ClassificationDataset(x_train,data_path = "",transform=transformer,training=True)
train_loader,val_loader = make_loader(dataset, train_batch_size=64, validation_split=0.2)

In [ ]:
test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                               torchvision.transforms.ToTensor(),
                                               transform.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

In [ ]:
testset = ClassificationDataset(x_test,data_path = "",transform=test_transform,training=False)
test_loader = torch.utils.data.DataLoader(
        testset, batch_size=1, shuffle=False,
    )

## 3. Define Optimizers and Loss function

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch import nn
import torch.nn.functional as F
criterion =  nn.CrossEntropyLoss()

In [ ]:
device

## 4. Define the Model: Transfer Learning

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.max_pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.max_pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.max_pool3 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(128, 128, 3)
        self.max_pool4 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(128*12*12, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.max_pool3(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.max_pool4(x)
        # Expand
        x = x.view(-1, 128*12*12)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        return x

In [ ]:
from torchvision import models
from torch import nn
model = SimpleCNN()



In [ ]:
model = model.to(device)
for data,t in train_loader:
    data = data.to(device)
    c = model(data)
    print(c.shape)
    break

### Sử dụng scheduler

#### Scheduler sẽ tự động giảm learning rate theo 1 hệ số (vd 0.5) khi val_loss không giảm / val_acc không tăng sau X epochs

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, "min", patience=2, factor=0.5
    )

## 5. Define a training epoch

In [ ]:
model = model.to(device)

In [ ]:
from sklearn.metrics import accuracy_score
def train_one_epoch(
    model,
    train_loader,
    test_loader,
    device,
    optimizer,
    criterion
):

    # training-the-model
    train_loss = 0
    valid_loss = 0
    all_labels = []
    all_preds = []
    model.train()
    for data, target in train_loader:
        # move-tensors-to-GPU
        data = data.type(torch.FloatTensor).to(device)
        # target=torch.Tensor(target)
        target = target.float().to(device)
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        # get the prediction label and target label
        output = model(data)
        preds = torch.argmax(output, axis=1).cpu().detach().numpy()
        labels = target.cpu().numpy()
        # calculate-the-batch-loss
        loss = criterion(output.type(torch.FloatTensor), target.type(torch.LongTensor))
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        # calculate training metrics
        all_labels.extend(labels)
        all_preds.extend(preds)
    
    acc = accuracy_score(all_labels, all_preds)

    # validate-the-model
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for data, target in test_loader:
            data = data.type(torch.FloatTensor).to(device)
            target = target.to(device)
            output = model(data)
            preds = torch.argmax(output, axis=1).tolist()
            labels = target.tolist()
            all_labels.extend(labels)
            all_preds.extend(preds)
            loss = criterion(output, target)

            # update-average-validation-loss
            valid_loss += loss.item() * data.size(0)

    val_acc = accuracy_score(all_labels, all_preds)
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(test_loader.sampler)

    return (
        train_loss,
        valid_loss,
        acc,
        val_acc,
    )

## 6. Begin Training

In [ ]:
from tqdm import tqdm

In [ ]:
!nvidia-smi

In [ ]:
num_epoch = 20
best_val_acc = 0
import logging
import numpy as np
print("begin training process")
for i in tqdm(range(0, num_epoch)):
    loss, val_loss, train_result, val_result = train_one_epoch(
        model,
        train_loader,
        val_loader,
        device,
        optimizer,
        criterion,
    )

    scheduler.step(val_loss)
    writer.add_scalar('Train/Loss', loss, i)
    writer.add_scalar('Val/Loss', val_loss, i)
    writer.add_scalar('Train/Acc', train_result, i)
    writer.add_scalar('Val/Acc', train_result, i)
    print(
        "Epoch {} / {} \n Training loss: {} -  training acc: {}".format(
            i + 1, num_epoch, loss,train_result
        )
    )
    print(
        " \n Validation loss : {} -  validation acc: {}".format(val_loss,val_result)
    )
    print("\n")
    # saving epoch with best validation accuracy
    if best_val_acc < float(val_result):
        print(
            "Validation accuracy= "+
            str(val_result)+
            "===> Save best epoch"
        )
        best_val_acc = val_result
        torch.save(
            model,
            "./" +  "best.pt",
        )
    else:
        print(
            "Validation accuracy= "+ str(val_result)+ "===> No saving"
        )
        continue

## Test the results

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
import copy
test_model = torch.load("/kaggle/working/best.pt")
test_model = test_model.to(device)

In [ ]:
def test_result(model, test_loader, device):
    # testing the model by turning model "Eval" mode
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for data, target in test_loader:
            # move-tensors-to-GPU
            data = data.to(device)
            target = target.to(device)
            # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
            output = model(data)
            prob = nn.Softmax(dim=1)
            # applying Softmax to results
            probs = prob(output)
            labels.extend(target.tolist())
            preds.extend(torch.argmax(probs, axis=1).tolist())
    return labels,preds


In [ ]:
labels,preds =test_result(test_model, test_loader, device)

In [ ]:
report = classification_report(labels, preds, digits=4,target_names=['cat','dog'])



In [ ]:
print(report)